# Use immutables for efficiency

Memory hierarchy crash course:

1. When CPU runs an instruction it operates on things in the registers. There are very few of these in a computer, these are the %1s and %2s when you run @code_llvm. If something is not in the CPU registers, the CPU needs to fetch the data from memory; this is **slow**.

2. The CPU first looks at L1 cache, then L2 cache then Main memory and then Swap space - L1, L2 caches are still small (order of megabytes), but hitting them often will give orders of magnitude performance gain as compared to hitting main memory that often

3. The computer optimistically brings things from main memory into the caches when you access a chunk of data. Hence if you access data that is contiguous in memory, they all get asynchronously brought into the cache and your program will be really fast.

## Memory layout of an array of `mutable struct` objects

In [ ]:
abstract type TestType end

In [ ]:
mutable struct Typ <: TestType
    x::Int16
    y::Int16
end

In [ ]:
Base.:+{T<:TestType}(a::T, b::T) = T(a.x+b.x, a.y+b.y)

In [ ]:
Typ(2,2) + Typ(4,4)

The size of `Typ` is 4 bytes:

In [ ]:
sizeof(Typ(2,2))

In [ ]:
@time typ_arr = [Typ(i%127,i%127) for i=1:10^6];

Notice the allocation. Also, this array is 2x bigger than it should be:

In [ ]:
sizeof(typ_arr)

In [ ]:
sizeof(typ_arr) / 10^6 # bytes per object

This is because mutable objects are **passed by reference**; the objects are being "boxed".

This is to make the following possible:

In [ ]:
function someone_else_doing_something_else(a::Typ)
    a.x = 42
end

someone_else_doing_something_else(typ_arr[3])
typ_arr[3]

Sum could also have been much more efficient....

In [ ]:
@time sum(typ_arr)

## Memory layout of an array of (immutable) `struct`s

In [ ]:
struct Imm <: TestType
    x::Int16
    y::Int16
end

In [ ]:
sizeof(Imm(2,2))

In [ ]:
@time imm_arr = [Imm(i%127, i%127) for i=1:10^6];

In [ ]:
sizeof(imm_arr)

This seems correct! Since immutables can never be changed, their value _is_ their identity, and the compiler can **pass them by value**. [But it does not necessarily.]

In [ ]:
Base.:+(a::Imm, b::Imm) = Imm(a.x+b.x, a.y+b.y)

In [ ]:
@time sum(typ_arr)

The allocation is the same as adding Float *values*

In [ ]:
x = rand(10^6)

@time sum(x)

The compiler can do this optimization because it knows someone else won't be changing the insides of the `Imm` object:

In [ ]:
function someone_else_doing_something_else(a::Imm)
    a.x = 42 # This is not allowed!!
end

someone_else_doing_something_else(imm_arr[3])

If you don't know the type of the insides of an immutable type, you can add a type parameter; e.g.

In [ ]:
immutable ImmParam{T} <: TestType
    x::T
    y::T
end

In [ ]:
sizeof(ImmParam{Int128}) # sizeof also works on the 

In [ ]:
sizeof(ImmParam{Int8})

In [ ]:
ImmParam{Int8} == ImmParam{Int64}

In [ ]:
ImmParam(1.0, 2.0) # Julia automatically infers this

In [ ]:
ImmParam(1,2)

In [ ]:
ImmParam(1.0,2)

### And it is aligned tightly!

In [ ]:
@time imm_par_array_int16 = [ImmParam{Int16}(2,3) for i = 1:10^6];

In [ ]:
sizeof(imm_par_array_int16)

In [ ]:
@time imm_par_array_int8 = [ImmParam{Int8}(2,3) for i = 1:10^6];

In [ ]:
sizeof(imm_par_array_int8)

In [ ]:
@time imm_par_array_cplx = [ImmParam(2+3im,3+2im) for i = 1:10^6];

In [ ]:
sizeof(imm_par_array_cplx)

In [ ]:
Base.:+(a::ImmParam, b::ImmParam) = ImmParam(a.x+b.x, a.y+b.y)

In [ ]:
@time sum(imm_par_array_cplx)

In [ ]:
using Interact

In [ ]:
type TypParam{T} <: TestType
    x::T
    y::T
end

In [ ]:
@manipulate for param = [Int8,Int16,Int32,Int64,Float16,Float32,Float64], complex=true
    T = complex ? Complex{param} : param
    a = zero(T)
    b = one(T)

    gc()  
    local arr,t_create,arr_t,t_create_t,t_sum,t_sum_t

    alloc_create = @allocated begin
        t_create = @elapsed begin
            arr = [ImmParam(a,b) for i=1:10^6]
        end
    end

    alloc_create_t = @allocated begin
        t_create_t = @elapsed begin
            arr_t = [TypParam(a,b) for i=1:10^6]
        end
    end

    gc()
    
    alloc_sum = @allocated begin
        t_sum = @elapsed begin
            s = sum(arr)
        end
    end/10^6

    alloc_sum_t = @allocated begin
        t_sum_t = @elapsed begin
            s_t = sum(arr_t)
        end
    end/10^6
    HTML("<table><tbody>
        <thead>
        <tr>
            <th colspan=4>mutable</th> 
            <th colspan=4>immutable</th>
        </tr>
        <tr>
            <th colspan=2>create</th> 
            <th colspan=2>sum</th>
            <th colspan=2>create</th> 
            <th colspan=2>sum</th>
        </tr>
        <tr>
            <th>time</th>
            <th>memory</th>
            <th>time</th>
            <th>memory</th>
            <th>time</th>
            <th>memory</th>
            <th>time</th>
            <th>memory</th>
        </tr>
        </thead>
        <tr>
            <td>$t_create</td>
            <td>$alloc_create</td>
            <td>$t_sum</td>
            <td>$alloc_sum</td>
            <td>$t_create_t</td>
            <td>$alloc_create_t</td>
            <td>$t_sum_t</td>
            <td>$alloc_sum_t</td>
        </tr>
        </tbody></table>")
end

## But be careful: vectors of heterogeneous types force boxing!

In [ ]:
["xyzabc", 1+2im, 1, 1.0]

In [ ]:
[ImmParam(UInt8(1),UInt8(1)), ImmParam(1.0,1.0)] |> sizeof

In [ ]:
@time heter_arr = [i%2 == 0 ? ImmParam(UInt8(1),UInt8(1)) : ImmParam(1.0,1.0) for i = 1:10^6]

In [ ]:
@time sum(heter_arr)

## Summary

- Use immutables wherever you consider something to be a *value*. Use type when something is a *state*.
- Never create a large array of mutable objects! Each one is heap-allocated, this kills performance and gives the GC a hard time.
- Parameterize if you need to change types
- In the wizard's own words: http://julialang.org/blog/2013/03/efficient-aggregates

## New packages that eliminate some of the drudgery

There are now packages available which allow us to get round some of these problems, e.g. https://github.com/tkoolen/TypeSortedCollections.jl

This automatically splits up a heterogeneous collection into a sequence of homogeneous collections, and provides a nice interface so that we do not have to the book-keeping ourselves.